In [6]:
import numpy as np
import pandas as pd

def load_data(file_path):
    data = pd.read_csv(file_path, header=None)
    X = data.iloc[:, :-1].values
    y = data.iloc[:, -1].values
    y = 2 * y - 1  # Convert labels to {1, -1}
    # Add bias term to feature vectors
    X = np.c_[np.ones(X.shape[0]), X]
    return X, y

def shuffle_data(X, y):
    indices = np.random.permutation(len(y))
    return X[indices], y[indices]

def hinge_loss(w, X, y, C):
    loss = 1 - y * np.dot(X, w)
    loss[loss < 0] = 0  # max(0, loss)
    regularizer = 0.5 * np.dot(w[1:], w[1:]) / C  # Exclude bias term from regularization
    return np.mean(loss) + regularizer

def subgradient(w, x, y, C):
    if 1 - y * np.dot(w, x) > 0:
        gradient = -y * x + np.concatenate(([0], w[1:])) / C  # Exclude bias term from gradient
    else:
        gradient = np.concatenate(([0], w[1:])) / C  # Exclude bias term from gradient
    return gradient

def svm_sgd(X_train, y_train, X_test, y_test, C, max_epochs, gamma_0, schedule_type):
    m, n = X_train.shape
    w = np.zeros(n)

    gamma = gamma_0

    for epoch in range(1, max_epochs + 1):
        X_train, y_train = shuffle_data(X_train, y_train)

        for i in range(m):
            xi, yi = X_train[i], y_train[i]
            gradient = subgradient(w, xi, yi, C)
            w = w - gamma * gradient

        # Update learning rate based on the schedule_type
        if schedule_type == "linear":
            gamma = gamma_0 / (1 + epoch)
        elif schedule_type == "exponential":
            gamma = gamma_0 / (1 + (((gamma_0 / 0.01) * epoch)))

    # Calculate final training and test errors
    train_error = hinge_loss(w, X_train, y_train, C)
    test_error = hinge_loss(w, X_test, y_test, C)

    return w, train_error, test_error

# Load data
train_file_path = "C:\\RSanthosh\\ML\\bank-note-1\\bank-note\\train.csv"
test_file_path = "C:\\Santhosh\\ML\\bank-note-1\\bank-note\\test.csv"
X_train, y_train = load_data(train_file_path)
X_test, y_test = load_data(test_file_path)

# Set hyperparameters
C_values = [100/873, 500/873, 700/873]
max_epochs = 100
gamma_0 = 0.1

# Run SVM for each C value with both learning rate schedules
for C in C_values:
    print(f"\nTraining SVM with C={C} using Learning rate gamma_0 / (1 + epoch)...\n")
    w_linear, train_error_linear, test_error_linear = svm_sgd(X_train, y_train, X_test, y_test, C, max_epochs, gamma_0, "linear")

    print(f"\nTraining SVM with C={C} using Learning rate gamma_0 / (1 + (((gamma_0 / 0.01) * epoch)))...\n")
    w_exp, train_error_exp, test_error_exp = svm_sgd(X_train, y_train, X_test, y_test, C, max_epochs, gamma_0, "exponential")

    # Calculate differences
    weights_diff = np.linalg.norm(w_linear - w_exp)
    bias_diff = np.abs(w_linear[0] - w_exp[0])
    train_error_diff = np.abs(train_error_linear - train_error_exp)
    test_error_diff = np.abs(test_error_linear - test_error_exp)

    # Print differences
    print(f"Weights Difference: {weights_diff}")
    print(f"Bias Parameter Difference: {bias_diff}")
    print(f"Final Training Error Difference: {train_error_diff}")
    print(f"Final Test Error Difference: {test_error_diff}")



Training SVM with C=0.1145475372279496 using Learning rate gamma_0 / (1 + epoch)...


Training SVM with C=0.1145475372279496 using Learning rate gamma_0 / (1 + (((gamma_0 / 0.01) * epoch)))...

Weights Difference: 0.027240405917548925
Bias Parameter Difference: 0.01040859862754212
Final Training Error Difference: 0.005054320648132049
Final Test Error Difference: 0.00879999055540015

Training SVM with C=0.572737686139748 using Learning rate gamma_0 / (1 + epoch)...


Training SVM with C=0.572737686139748 using Learning rate gamma_0 / (1 + (((gamma_0 / 0.01) * epoch)))...

Weights Difference: 0.025373526976307216
Bias Parameter Difference: 0.003922748089443595
Final Training Error Difference: 0.011437181099014981
Final Test Error Difference: 0.008641982013275873

Training SVM with C=0.8018327605956472 using Learning rate gamma_0 / (1 + epoch)...


Training SVM with C=0.8018327605956472 using Learning rate gamma_0 / (1 + (((gamma_0 / 0.01) * epoch)))...

Weights Difference: 0.03003878581